```
-- 3
-- Write a query that returns customer and employee pairs 
-- that had order activity in January 2016 but not in February 2016
-- Tables involved: TSQLV4 database, Orders table

--Desired output
custid      empid
----------- -----------
1           1
3           3
5           8
5           9
6           9
7           6
9           1
12          2
16          7
17          1
20          7
24          8
25          1
26          3
32          4
38          9
39          3
40          2
41          2
42          2
44          8
47          3
47          4
47          8
49          7
55          2
55          3
56          6
59          8
63          8
64          9
65          3
65          8
66          5
67          5
70          3
71          2
75          1
76          2
76          5
80          1
81          1
81          3
81          4
82          6
84          1
84          3
84          4
88          7
89          4

```

<span class="mi" style="color: rgb(102, 102, 102);"><br></span>

In [ ]:
USE TSQLV4;

-- Start by selecting distinct pairs of customer and employee IDs
SELECT DISTINCT 
    Orders.custid, -- Select the customer ID
    Orders.empid   -- Select the employee ID
FROM 
    Sales.Orders  -- Specify the Orders table from the TSQLV4 database
WHERE 
    -- Filter orders to include only those made in January 2016
    Orders.orderdate >= '2016-01-01' -- Orders made on or after January 1, 2016
    AND 
    Orders.orderdate < '2016-02-01'  -- But before February 1, 2016
    AND
    -- Exclude pairs that also had orders in February 2016
    NOT EXISTS (
        -- Subquery to identify orders in February 2016
        SELECT 1 -- A constant value; used because EXISTS just checks for the existence of rows
        FROM Sales.Orders as FebOrders -- Alias the Orders table to differentiate from the outer query
        WHERE
            -- Match customer ID in the subquery with the outer query
            FebOrders.custid = Orders.custid
            AND 
            -- Match employee ID in the subquery with the outer query
            FebOrders.empid = Orders.empid
            AND
            -- Filter for orders in February 2016
            FebOrders.orderdate >= '2016-02-01' -- Orders on or after February 1, 2016
            AND 
            FebOrders.orderdate < '2016-03-01'  -- But before March 1, 2016
    );

### Proposition:

- The query aims to find distinct pairs of customers and employees (`custid`, `empid`) from the `Sales.Orders` table who had order activity in January 2016 but not in February 2016.

### Tables involved:

- `Sales.Orders` (used twice, once as itself and once as the alias `FebOrders` in the subquery)

### Columns:

- `Orders.custid`: Customer ID from the Orders table
- `Orders.empid`: Employee ID from the Orders table
- `FebOrders.custid`: Customer ID from the subquery alias for the Orders table (for February orders)
- `FebOrders.empid`: Employee ID from the subquery alias for the Orders table (for February orders)
- `Orders.orderdate`: Order date from the Orders table
- `FebOrders.orderdate`: Order date from the subquery alias for the Orders table (for February orders)

### Predicate:

- The main query selects distinct customer and employee pairs (`custid`, `empid`) from the Orders table.
- The `WHERE` clause includes two parts:
    1. Filtering orders made in January 2016:
        - `Orders.orderdate >= '2016-01-01'`: Includes orders on or after January 1, 2016.
        - `Orders.orderdate < '2016-02-01'`: Excludes orders on or after February 1, 2016.
    2. Using `NOT EXISTS` to exclude pairs that had orders in February 2016. This is done by a subquery:
        - The subquery checks for the existence of orders for the same customer and employee pairs in February 2016.
        - Predicate for the subquery:
            - `FebOrders.orderdate >= '2016-02-01'`: Includes orders on or after February 1, 2016.
            - `FebOrders.orderdate < '2016-03-01'`: Excludes orders on or after March 1, 2016.
        - The `NOT EXISTS` clause ensures that the pairs selected from the main query do not have a match in the subquery.

```
-- 4
-- Write a query that returns customer and employee pairs 
-- that had order activity in both January 2016 and February 2016
-- Tables involved: TSQLV4 database, Orders table

--Desired output
custid      empid
----------- -----------
20          3
39          9
46          5
67          1
71          4

```

<span class="mi" style="color: rgb(102, 102, 102);"><br></span>

In [ ]:
USE TSQLV4;

SELECT DISTINCT 
    JanOrders.custid, -- Select the customer ID from January orders
    JanOrders.empid   -- Select the employee ID from January orders
FROM 
    Sales.Orders AS JanOrders -- Alias the Orders table as JanOrders for January orders
WHERE 
    -- Filter January orders to include only those made in January 2016
    JanOrders.orderdate >= '2016-01-01' -- Orders made on or after January 1, 2016
    AND 
    JanOrders.orderdate < '2016-02-01'  -- But before February 1, 2016
    AND
    -- Ensure pairs also had orders in February 2016
    EXISTS (
        -- Subquery to identify orders in February 2016
        SELECT 1 -- A constant value; used because EXISTS just checks for the existence of rows
        FROM Sales.Orders as FebOrders -- Alias the Orders table as FebOrders for February orders
        WHERE
            -- Match customer ID in the subquery with the January orders
            FebOrders.custid = JanOrders.custid
            AND 
            -- Match employee ID in the subquery with the January orders
            FebOrders.empid = JanOrders.empid
            AND
            -- Filter for orders in February 2016
            FebOrders.orderdate >= '2016-02-01' -- Orders on or after February 1, 2016
            AND 
            FebOrders.orderdate < '2016-03-01'  -- But before March 1, 2016
    );


### Proposition:

- The query is designed to identify distinct pairs of customers and employees (`custid`, `empid`) who had order activity in both January 2016 and February 2016.

### Tables involved:

- `Sales.Orders` (used twice with different aliases: `JanOrders` for January and `FebOrders` for February)

### Columns:

- `JanOrders.custid`: Customer ID from January orders
- `JanOrders.empid`: Employee ID from January orders
- `FebOrders.custid`: Customer ID from February orders in the subquery
- `FebOrders.empid`: Employee ID from February orders in the subquery
- `JanOrders.orderdate`: Order date from January orders
- `FebOrders.orderdate`: Order date from February orders in the subquery

### Predicate:

- The query selects distinct pairs of customer and employee IDs (`custid`, `empid`) from orders made in January 2016.
- The `WHERE` clause filters orders to those made in January 2016 (`JanOrders.orderdate` between '2016-01-01' and '2016-02-01').
- The `EXISTS` clause is used to ensure that for each pair from January, there is at least one matching order in February 2016. This is checked in a subquery (`FebOrders`) with the same customer and employee IDs and order dates in February 2016 (`FebOrders.orderdate` between '2016-02-01' and '2016-03-01').

In [ ]:
USE TSQLV4;
-- The INTERSECT ALL Multiset Operator (Optional, Advanced)
SELECT
  ROW_NUMBER() 
    OVER(PARTITION BY country, region, city
         ORDER     BY (SELECT 0)) AS rownum,
  country, region, city
FROM HR.Employees

INTERSECT

SELECT
  ROW_NUMBER() 
    OVER(PARTITION BY country, region, city
         ORDER     BY (SELECT 0)),
  country, region, city
FROM Sales.Customers;

<span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);">Can you identify the shared geographic locations between our employees and customers? We're interested in pinpointing specific countries, regions, and cities where both our employees and customers are located, considering each unique combination of these geographical attributes."</span>

<span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);"><br></span>

<span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);">This question uses an </span> `INTERSECT`  <span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);"> operation between the </span> `HR.Employees` <span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);">and </span> `Sales.Customers` <span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);">tables. It is focused on finding common geographic locations, considering each unique combination of country, region, and city.</span><span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);"><br></span>

In [ ]:
USE TSQLV4;
WITH INTERSECT_ALL
AS
(
  SELECT
    ROW_NUMBER() 
      OVER(PARTITION BY country, region, city
           ORDER     BY (SELECT 0)) AS rownum,
    country, region, city
  FROM HR.Employees

  INTERSECT

  SELECT
    ROW_NUMBER() 
      OVER(PARTITION BY country, region, city
           ORDER     BY (SELECT 0)),
    country, region, city
  FROM Sales.Customers
)
SELECT country, region, city
FROM INTERSECT_ALL;

<span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);">Could you provide a refined list of geographical areas where both our employees and customers are present, ensuring that we account for each unique occurrence of these locations in our records? We need this data structured for further detailed analysis.</span>

<span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);">This question aligns with the second query that utilizes a Common Table Expression (CTE) with an </span> `INTERSECT` <span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);">operation. The use of a CTE suggests a need for a more structured approach, possibly for further processing or analysis. This query also focuses on identifying shared locations, but the structure implies a more complex analytical requirement or a step in a larger data processing workflow.</span><span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);"><br></span>

In [ ]:
USE TSQLV4;
-- The EXCEPT Distinct Set Operator

-- Employees EXCEPT Customers
SELECT country, region, city FROM HR.Employees
EXCEPT
SELECT country, region, city FROM Sales.Customers;

<span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);">"Can you identify the geographical locations (country, region, city) where we have employees but no customers? We're looking to explore potential new markets or areas for customer outreach, focusing on regions where we already have an employee presence."</span>

<span style="font-size: 16px; white-space-collapse: preserve;"><br></span>

<span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);">This query finds locations that are exclusive to the </span> `HR.Employees` <span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);"> table, meaning these locations are not present in the </span> `Sales.Customers` <span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);"> table. The result would help in identifying potential new market areas or regions where the company might focus its customer acquisition efforts.</span><span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);"><br></span>

In [ ]:
USE TSQLV4;
-- Customers EXCEPT Employees
SELECT country, region, city FROM Sales.Customers
EXCEPT
SELECT country, region, city FROM HR.Employees;

<span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);">"Could you provide a list of geographical locations (country, region, city) where we have customers but no employees? We want to identify areas where our customer base is present, but we lack an employee footprint, for potential expansion of our operational staff."</span>

<span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);"><br></span>

<span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);">This query identifies locations that are exclusive to the </span> `Sales.Customers` <span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);"> table, meaning these locations are not present in the </span> `HR.Employees` <span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);"> table. The results will highlight areas where the company has a customer presence but currently does not have any employees, which could be crucial for strategic planning, especially in terms of expanding staff or establishing new offices.</span><span style="color: rgb(13, 13, 13); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve; background-color: rgb(255, 255, 255);"><br></span>

Written in collaboration with **ChatGPT** from **OpenAI** to improve understanding and assist with the explanation of the query.